# JIT-Optimized Packing with Backward Propagation

This experiment implements:
1. Numba JIT-compiled scoring functions
2. Fix direction rotation optimization
3. Backward propagation from n=200 to n=1
4. Local search with fine-grained moves

In [1]:
import math
import numpy as np
import pandas as pd
from numba import njit
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
import time
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x = np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2], np.float64)
    y = np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1], np.float64)
    return x, y

# Fast score calculation using JIT
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score = side^2 / n for a group of trees."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

@njit
def get_side_length_jit(xs, ys, degs, tx, ty):
    """Get the side length of bounding box."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

# Warm up JIT
tx, ty = make_polygon_template()
test_xs = np.array([0.0, 0.5], np.float64)
test_ys = np.array([0.0, 0.5], np.float64)
test_degs = np.array([0.0, 45.0], np.float64)
_ = score_group(test_xs, test_ys, test_degs, tx, ty)
_ = get_side_length_jit(test_xs, test_ys, test_degs, tx, ty)
print("JIT functions compiled")

JIT functions compiled


In [3]:
def strip(a):
    """Convert string values to float array."""
    return np.array([float(str(v).replace("s","")) for v in a], np.float64)

def load_submission(filepath):
    """Load submission and return dict of (xs, ys, degs) arrays per n."""
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        degs = strip(g['deg'].to_numpy())
        configs[n] = (xs, ys, degs)
    return configs

def calculate_total_score(configs, tx, ty):
    """Calculate total score across all configurations."""
    total = 0.0
    for n, (xs, ys, degs) in configs.items():
        total += score_group(xs, ys, degs, tx, ty)
    return total

print("Helper functions defined")

Helper functions defined


In [4]:
# Load best submission
print("Loading best submission...")
configs = load_submission('/home/code/experiments/002_improved_optimizer/submission.csv')
initial_score = calculate_total_score(configs, tx, ty)
print(f"Initial score: {initial_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {initial_score - 68.922808:.6f}")

Loading best submission...
Initial score: 146.566822
Target: 68.922808
Gap: 77.644014


In [5]:
# Fix direction rotation optimization
def calculate_bbox_side_at_angle(angle_deg, hull_points):
    """Calculate bounding box side at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = hull_points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def fix_direction(xs, ys, degs, tx, ty):
    """Rotate entire configuration to minimize bounding box."""
    # Get all vertices
    all_points = []
    for i in range(len(xs)):
        r = degs[i] * np.pi / 180.0
        c, s = np.cos(r), np.sin(r)
        for j in range(len(tx)):
            X = c*tx[j] - s*ty[j] + xs[i]
            Y = s*tx[j] + c*ty[j] + ys[i]
            all_points.append([X, Y])
    points = np.array(all_points)
    
    # Get convex hull for efficiency
    try:
        hull_points = points[ConvexHull(points).vertices]
    except:
        hull_points = points
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Find optimal rotation angle
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), method='bounded'
    )
    
    if res.fun < initial_side - 1e-8:
        # Apply rotation
        angle_deg = res.x
        angle_rad = np.radians(angle_deg)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        
        # Find center
        cx = np.mean(xs)
        cy = np.mean(ys)
        
        # Rotate positions
        new_xs = c * (xs - cx) - s * (ys - cy) + cx
        new_ys = s * (xs - cx) + c * (ys - cy) + cy
        new_degs = degs + angle_deg
        
        return new_xs, new_ys, new_degs, res.fun
    else:
        return xs, ys, degs, initial_side

print("Fix direction function defined")

Fix direction function defined


In [6]:
# Apply fix_direction to all configurations
print("Applying fix_direction to all configurations...")
start_time = time.time()

for n in range(1, 201):
    xs, ys, degs = configs[n]
    new_xs, new_ys, new_degs, new_side = fix_direction(xs, ys, degs, tx, ty)
    configs[n] = (new_xs, new_ys, new_degs)

fix_dir_score = calculate_total_score(configs, tx, ty)
print(f"After fix_direction: {fix_dir_score:.6f}")
print(f"Improvement: {initial_score - fix_dir_score:.6f}")
print(f"Time: {time.time() - start_time:.1f}s")

Applying fix_direction to all configurations...


After fix_direction: 146.566822
Improvement: 0.000000
Time: 0.4s


In [7]:
# Backward propagation
def backward_propagation(configs, tx, ty):
    """Improve smaller configs by removing trees from larger ones."""
    improvements = 0
    
    for n in range(199, 0, -1):  # From 199 down to 1
        current_xs, current_ys, current_degs = configs[n]
        current_side = get_side_length_jit(current_xs, current_ys, current_degs, tx, ty)
        current_score = current_side ** 2 / n
        
        # Get n+1 configuration
        next_xs, next_ys, next_degs = configs[n + 1]
        
        # Try removing each tree from n+1 config
        best_score = current_score
        best_config = (current_xs, current_ys, current_degs)
        
        for i in range(n + 1):
            # Remove tree i
            test_xs = np.delete(next_xs, i)
            test_ys = np.delete(next_ys, i)
            test_degs = np.delete(next_degs, i)
            
            test_side = get_side_length_jit(test_xs, test_ys, test_degs, tx, ty)
            test_score = test_side ** 2 / n
            
            if test_score < best_score - 1e-10:
                best_score = test_score
                best_config = (test_xs.copy(), test_ys.copy(), test_degs.copy())
        
        if best_score < current_score - 1e-10:
            configs[n] = best_config
            improvements += 1
    
    return improvements

print("Backward propagation function defined")

Backward propagation function defined


In [8]:
# Apply backward propagation
print("Applying backward propagation...")
start_time = time.time()

for iteration in range(3):  # Multiple passes
    improvements = backward_propagation(configs, tx, ty)
    bp_score = calculate_total_score(configs, tx, ty)
    print(f"Pass {iteration+1}: {improvements} improvements, score={bp_score:.6f}")
    if improvements == 0:
        break

print(f"After backward propagation: {bp_score:.6f}")
print(f"Time: {time.time() - start_time:.1f}s")

Applying backward propagation...


Pass 1: 185 improvements, score=133.130856


Pass 2: 0 improvements, score=133.130856
After backward propagation: 133.130856
Time: 0.5s


In [10]:
# Local search with JIT - fixed version
@njit
def local_search_jit(xs, ys, degs, tx, ty, max_iterations=100):
    """Fast local search using JIT."""
    n = xs.size
    if n <= 1:
        return xs.copy(), ys.copy(), degs.copy()
    
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_degs = degs.copy()
    best_side = get_side_length_jit(best_xs, best_ys, best_degs, tx, ty)
    
    step_sizes = np.array([0.01, 0.005, 0.002, 0.001, 0.0005])
    rot_deltas = np.array([5.0, 2.0, 1.0, 0.5])
    directions = np.array([[-1, 0], [1, 0], [0, -1], [0, 1], [-1, -1], [-1, 1], [1, -1], [1, 1]])
    
    for iteration in range(max_iterations):
        improved = False
        
        for i in range(n):
            # Try translation moves
            for step_idx in range(len(step_sizes)):
                step = step_sizes[step_idx]
                for dir_idx in range(len(directions)):
                    dx = directions[dir_idx, 0] * step
                    dy = directions[dir_idx, 1] * step
                    
                    test_xs = best_xs.copy()
                    test_ys = best_ys.copy()
                    test_xs[i] += dx
                    test_ys[i] += dy
                    
                    test_side = get_side_length_jit(test_xs, test_ys, best_degs, tx, ty)
                    if test_side < best_side - 1e-10:
                        best_xs = test_xs
                        best_ys = test_ys
                        best_side = test_side
                        improved = True
            
            # Try rotation moves
            for rot_idx in range(len(rot_deltas)):
                delta = rot_deltas[rot_idx]
                for sign in [-1.0, 1.0]:
                    d_angle = sign * delta
                    test_degs = best_degs.copy()
                    test_degs[i] += d_angle
                    
                    test_side = get_side_length_jit(best_xs, best_ys, test_degs, tx, ty)
                    if test_side < best_side - 1e-10:
                        best_degs = test_degs
                        best_side = test_side
                        improved = True
        
        if not improved:
            break
    
    return best_xs, best_ys, best_degs

# Warm up
_ = local_search_jit(test_xs, test_ys, test_degs, tx, ty, max_iterations=1)
print("Local search JIT compiled")

Local search JIT compiled


In [11]:
# Apply local search to worst-performing configurations
print("Applying local search to configurations...")
start_time = time.time()

# Calculate score contribution for each n
scores = {}
for n, (xs, ys, degs) in configs.items():
    scores[n] = score_group(xs, ys, degs, tx, ty)

# Sort by score contribution (highest first)
sorted_n = sorted(scores.keys(), key=lambda n: scores[n], reverse=True)

# Apply local search to top 50 worst configs
for idx, n in enumerate(sorted_n[:50]):
    xs, ys, degs = configs[n]
    new_xs, new_ys, new_degs = local_search_jit(xs, ys, degs, tx, ty, max_iterations=50)
    configs[n] = (new_xs, new_ys, new_degs)
    
    if (idx + 1) % 10 == 0:
        current_score = calculate_total_score(configs, tx, ty)
        print(f"Processed {idx+1}/50, score={current_score:.6f}")

ls_score = calculate_total_score(configs, tx, ty)
print(f"After local search: {ls_score:.6f}")
print(f"Time: {time.time() - start_time:.1f}s")

Applying local search to configurations...


Processed 10/50, score=129.839209


Processed 20/50, score=128.003951


Processed 30/50, score=126.562092


Processed 40/50, score=125.360608


Processed 50/50, score=124.281862
After local search: 124.281862
Time: 60.1s


In [ ]:
# Final fix_direction pass
print("Final fix_direction pass...")
for n in range(1, 201):
    xs, ys, degs = configs[n]
    new_xs, new_ys, new_degs, _ = fix_direction(xs, ys, degs, tx, ty)
    configs[n] = (new_xs, new_ys, new_degs)

final_score = calculate_total_score(configs, tx, ty)
print(f"Final score: {final_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {final_score - 68.922808:.6f}")
print(f"Total improvement: {initial_score - final_score:.6f}")

In [ ]:
# Validate using Shapely
class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Validating configurations...")
invalid = []
for n, (xs, ys, degs) in configs.items():
    trees = [ChristmasTree(xs[i], ys[i], degs[i]) for i in range(n)]
    if has_overlap(trees):
        invalid.append(n)

if invalid:
    print(f"WARNING: {len(invalid)} configs have overlaps: {invalid[:10]}...")
else:
    print("All configurations valid (no overlaps)")

In [ ]:
# Save submission
print("Saving submission...")

rows = []
for n in range(1, 201):
    xs, ys, degs = configs[n]
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{xs[i]:.6f}',
            'y': f's{ys[i]:.6f}',
            'deg': f's{degs[i]:.6f}'
        })

submission = pd.DataFrame(rows)
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/006_jit_optimizer/submission.csv', index=False)

print(f"Submission saved!")
print(f"Final Score: {final_score:.6f}")